In [19]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn


DEFAULT_PATH_DATA_CREDIT = r'E:\Hinevics\Dropbox\Programm\GitHub\Credit-Train\Data\credit_train.csv'

data = pd.read_csv(filepath_or_buffer=DEFAULT_PATH_DATA_CREDIT, encoding='utf-8', sep=';', index_col='client_id')

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (14) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,gender,age,marital_status,job_position,credit_sum,credit_month,tariff_id,score_shk,education,living_region,monthly_income,credit_count,overdue_credit_count,"open_account_flg,,"
client_id,,,,,,,,,,,,,,
1,M,NaN,NaN,UMN,"59998,00",10,1.6,NaN,GRD,КРАСНОДАРСКИЙ КРАЙ,30000.0,1.0,1.0,"0,"
2,F,NaN,MAR,UMN,"10889,00",6,1.1,NaN,NaN,МОСКВА,NaN,2.0,0.0,"0,"
3,M,32.0,MAR,SPC,"10728,00",12,1.1,NaN,NaN,ОБЛ САРАТОВСКАЯ,NaN,5.0,0.0,"0,"
4,F,27.0,NaN,SPC,"12009,09",12,1.1,NaN,NaN,ОБЛ ВОЛГОГРАДСКАЯ,NaN,2.0,0.0,"0,"
5,M,45.0,NaN,SPC,NaN,10,1.1,"0,421385",SCH,ЧЕЛЯБИНСКАЯ ОБЛАСТЬ,NaN,1.0,0.0,"0,"
